## Instalação das Bibliotecas

In [ ]:
!pip install pandas requests plotly

## Importação das bibliotecas

In [ ]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objs as go

## Função para coletar dados do Banco Central do Brasil

In [ ]:

def bcb_data(serie, start="01/01/2010", end=None):
    """Busca dados da API do Banco Central do Brasil.
    
    Parametros:
        serie (str | int): codigo da serie (ex. 4189 para Selic meta).
        start (str): data inicial dd/mm/aaaa.
        end (str | None): data final dd/mm/aaaa.
    """
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie}/dados"
    params = {"formato": "json", "dataInicial": start}
    if end:
        params["dataFinal"] = end
    r = requests.get(url, params=params, timeout=30)
    if r.status_code != 200:
        raise ValueError(f"Erro {r.status_code} ao acessar {url}")
    df = pd.DataFrame(r.json())
    df["data"] = pd.to_datetime(df["data"], dayfirst=True)
    df["valor"] = pd.to_numeric(df["valor"], errors="coerce")
    df = df.dropna()
    return df


## Função para coletar dados do FRED (EUA)

In [ ]:

def fred_data(series_id, api_key, start="2010-01-01", end=None):
    """Busca dados de uma série do FRED.
    
    Parametros:
        series_id (str): identificador da série no FRED.
        api_key (str): chave de acesso.
        start (str): data inicial AAAA-MM-DD.
        end (str | None): data final AAAA-MM-DD.
    """
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json",
        "observation_start": start
    }
    if end:
        params["observation_end"] = end
    r = requests.get(url, params=params, timeout=30)
    if r.status_code != 200:
        raise ValueError(f"Erro {r.status_code} ao acessar {url}")
    data = r.json().get("observations", [])
    if not data:
        raise ValueError("Nenhum dado retornado")
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df = df.dropna()
    return df


## Obtenção dos dados

In [ ]:

FRED_API_KEY = "COLE_SUA_API_KEY_AQUI"  # substitua pela sua chave FRED

selic = bcb_data(4189, start="2015-01-01")
fedfunds = fred_data("FEDFUNDS", FRED_API_KEY, start="2015-01-01")

ipca = bcb_data(433, start="2015-01-01")
cpi = fred_data("CPIAUCSL", FRED_API_KEY, start="2015-01-01")

dolar_bcb = bcb_data(1, start="2015-01-01")
dolar_fred = fred_data("DEXBZUS", FRED_API_KEY, start="2015-01-01")


## Alinhamento temporal dos dados

In [ ]:

df_rates = pd.merge(
    selic.resample("M", on="data").last().rename(columns={"valor": "selic"}),
    fedfunds.resample("M", on="date").last().rename(columns={"value": "fedfunds"}),
    left_index=True,
    right_index=True,
    how="inner"
)

df_inflation = pd.merge(
    ipca.rename(columns={"data": "date", "valor": "ipca"}),
    cpi.rename(columns={"value": "cpi"}),
    on="date",
    how="inner"
)

df_fx = pd.merge(
    dolar_bcb.resample("D", on="data").last().rename(columns={"valor": "brl_bcb"}),
    dolar_fred.resample("D", on="date").last().rename(columns={"value": "brl_fred"}),
    left_index=True,
    right_index=True,
    how="inner"
)


## Gráficos Interativos

In [ ]:

fig_rates = go.Figure()
fig_rates.add_trace(go.Scatter(x=df_rates.index, y=df_rates['selic'], mode='lines', name='Selic (%)'))
fig_rates.add_trace(go.Scatter(x=df_rates.index, y=df_rates['fedfunds'], mode='lines', name='Fed Funds (%)'))
fig_rates.update_layout(title='Taxas de Juros Brasil x EUA', xaxis_title='Data', yaxis_title='Taxa (%)')

fig_inflation = go.Figure()
fig_inflation.add_trace(go.Bar(x=df_inflation['date'], y=df_inflation['ipca'], name='IPCA'))
fig_inflation.add_trace(go.Bar(x=df_inflation['date'], y=df_inflation['cpi'], name='CPI'))
fig_inflation.update_layout(barmode='group', title='Inflação Mensal', xaxis_title='Data', yaxis_title='Variação (%)')

fig_fx = go.Figure()
fig_fx.add_trace(go.Scatter(x=df_fx.index, y=df_fx['brl_bcb'], mode='lines', name='BCB USD/BRL'))
fig_fx.add_trace(go.Scatter(x=df_fx.index, y=df_fx['brl_fred'], mode='lines', name='FRED USD/BRL'))
fig_fx.update_layout(title='Taxa de câmbio USD/BRL', xaxis_title='Data', yaxis_title='Reais por Dólar')

fig_rates.show()
fig_inflation.show()
fig_fx.show()


## Insights Automáticos (exemplo simples)

In [ ]:

ultimo_selic = df_rates['selic'].iloc[-1]
ultimo_fed = df_rates['fedfunds'].iloc[-1]
if ultimo_selic > ultimo_fed:
    print('A Selic está acima do Fed Funds, indicando política monetária mais apertada no Brasil.')
else:
    print('O Fed Funds está acima da Selic, indicando política monetária mais apertada nos EUA.')
